In [3]:
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

import os
import fnmatch
import re


import numpy as np
from CIFAKEClassifier import CIFAKEClassifier
from device import fetchDevice
import dataloader 
from dataloader import CIFAKEDataset


class DatasetClassifier:
    def __init__(self, fakeFolder):
        self.fakeFolder = fakeFolder
        self.data_set = CIFAKEDataset(fakeFolder=self.fakeFolder, num_processes=4)
        self.model = CIFAKEClassifier()
        self.model = self.model.to(fetchDevice())

        # Hyperparameters
        self.batch_size = 64
        self.learning_rate = 1e-3
        self.epochs = 5

        # Create the training and testing splits
        self.train_size = int(50_000)
        self.test_size = len(self.data_set) - self.train_size
        self.train_dataset, self.test_dataset = random_split(self.data_set, [self.train_size, self.test_size])


    def train(self):
        # Dataloader for batch training
        train_loader = DataLoader(dataset=self.train_dataset, batch_size=self.batch_size, shuffle=True)
        #self.test_loader = DataLoader(dataset=self.test_dataset, batch_size=self.batch_size, shuffle=False)

        # Loss function and optimizer
        criterion = nn.BCELoss()  # Binary Cross Entropy Loss for binary classification
        optimizer = Adam(self.model.parameters(), lr=self.learning_rate)
        self.model.train()
        for epoch in range(self.epochs):
            running_loss = 0.0
            for i, data in enumerate(train_loader, 0):
                inputs, labels = data
                labels = labels.float()  # BCELoss expects labels to be in float format

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = self.model(inputs).squeeze()  # Remove unnecessary dimensions
                loss = criterion(outputs, labels)

                # Backward pass and optimize
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                if i % 10 == 9:  # Print every 10 mini-batches
                    print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 10:.4f}')
                    running_loss = 0.0

        torch.save(self.model.state_dict(), 'weights/' + self.fakeFolder + '_model.pth')
        print('Finished Training ' + self.fakeFolder)

    def loadWeight(self, weightPath):
        self.model.load_state_dict(torch.load(weightPath))

    def fetchTestDataSet(self):
        return DataLoader(dataset=self.test_dataset, batch_size=self.batch_size, shuffle=False)
    
    
    def test_model(self, datasetClassifier):
        self.model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in datasetClassifier.fetchTestDataSet():
                images, labels = data
                outputs = self.model(images).squeeze()  # Remove unnecessary dimensions
                predicted = torch.round(outputs)  # Round to get binary predictions
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        return accuracy
        
    def getName(self):
        return self.fakeFolder

SyntaxError: invalid syntax. Perhaps you forgot a comma? (53832557.py, line 16)

In [2]:
fakeDatasets = list(os.listdir("STABLEDIFFUSION"))
datasetClassifiers = [0]*len(fakeDatasets)
counter = 0
for i in fakeDatasets:
    if i[0] == '.':
        continue
    datasetClassifiers[counter] = DatasetClassifier(f"STABLEDIFFUSION/{i}")
    datasetClassifiers[counter].train()
    counter += 1
    print(f"""\n
        !!!!!!!!!!!!!!!!!!!!!!
        CURRENTLY AT {counter}
        !!!!!!!!!!!!!!!!!!!!!!
        \n""")
    



Loading folder: CIFAKE/test/REAL


100%|██████████| 10000/10000 [00:01<00:00, 7192.35it/s]


Loading folder: CIFAKE/train/REAL


100%|██████████| 50000/50000 [00:07<00:00, 6976.06it/s]


Loading folder: STABLEDIFFUSION/sd21-rpoa-n-32jpg/SD21Bird


...

Loading folder: STABLEDIFFUSION/sd30cifake-32jpg/SD30Truck


100%|██████████| 6000/6000 [00:00<00:00, 8000.67it/s]


Loading folder: STABLEDIFFUSION/sd30cifake-32jpg/SD30Frog


100%|██████████| 6000/6000 [00:00<00:00, 7810.70it/s]


Epoch 1, Batch 10, Loss: 0.6907
Epoch 1, Batch 20, Loss: 0.6973
Epoch 1, Batch 30, Loss: 0.6878
Epoch 1, Batch 40, Loss: 0.6751
Epoch 1, Batch 50, Loss: 0.6335
Epoch 1, Batch 60, Loss: 0.5791
Epoch 1, Batch 70, Loss: 0.4999
Epoch 1, Batch 80, Loss: 0.4345
Epoch 1, Batch 90, Loss: 0.4372
Epoch 1, Batch 100, Loss: 0.3808
Epoch 1, Batch 110, Loss: 0.3627
Epoch 1, Batch 120, Loss: 0.3142
Epoch 1, Batch 130, Loss: 0.3516
Epoch 1, Batch 140, Loss: 0.3516
Epoch 1, Batch 150, Loss: 0.2886
Epoch 1, Batch 160, Loss: 0.2864
Epoch 1, Batch 170, Loss: 0.3165
Epoch 1, Batch 180, Loss: 0.3521
Epoch 1, Batch 190, Loss: 0.3601
Epoch 1, Batch 200, Loss: 0.2868
Epoch 1, Batch 210, Loss: 0.2984
Epoch 1, Batch 220, Loss: 0.2845
Epoch 1, Batch 230, Loss: 0.2814
Epoch 1, Batch 240, Loss: 0.2531
Epoch 1, Batch 250, Loss: 0.2714
Epoch 1, Batch 260, Loss: 0.2572
Epoch 1, Batch 270, Loss: 0.2466
Epoch 1, Batch 280, Loss: 0.3063
Epoch 1, Batch 290, Loss: 0.3337
Epoch 1, Batch 300, Loss: 0.2720
Epoch 1, Batch 310,

In [7]:
for dc in range(len(datasetClassifiers) - 1):
    #print
    print(str(dc) + " " + datasetClassifiers[dc].getName())

0 STABLEDIFFUSION/sd21-rpoa-n-32jpg
1 STABLEDIFFUSION/sd21cifake-32pxjpg-n
2 STABLEDIFFUSION/sd21-rpoa-32jpg
3 STABLEDIFFUSION/lora-sd21-v3-32JPG
4 STABLEDIFFUSION/lora-sd21-v1-32jpg
5 STABLEDIFFUSION/sd21-gpt40p-32jpg
6 STABLEDIFFUSION/sd21-apoa-32jpg
7 STABLEDIFFUSION/d21cifake-512pxjpg
8 STABLEDIFFUSION/sd21cifake-32jpg
9 STABLEDIFFUSION/lora-sd21-v2-32jpg
10 STABLEDIFFUSION/sd21-apoan-32jpg
11 STABLEDIFFUSION/sd21cifake-512pxjpg-n
12 STABLEDIFFUSION/sd30-dmp32jpg
13 STABLEDIFFUSION/sd30cifake-32jpg


In [16]:
datasetClassifiers[13].test_model(datasetClassifiers[13])

95.51857142857143